In [20]:
import pandas as pd
import os
import numpy as np

abs_path = os.path.abspath("")



## 510k

In [21]:
'''Searching functions are ready, there are 10 parameters for filtering: ['KNUMBER', 'APPLICANT', "COUNTRY_CODE", "DATERECEIVED", 'DECISIONDATE', "DECISION", 'PRODUCTCODE', "STATEORSUMM", "THIRDPARTY", 'DEVICENAME']. Only  "DATERECEIVED", 'DECISIONDATE' take list as parameters, [year, mode]'''

'Searching functions are ready, there are 10 parameters for filtering: [\'KNUMBER\', \'APPLICANT\', "COUNTRY_CODE", "DATERECEIVED", \'DECISIONDATE\', "DECISION", \'PRODUCTCODE\', "STATEORSUMM", "THIRDPARTY", \'DEVICENAME\']. Only  "DATERECEIVED", \'DECISIONDATE\' take list as parameters, [year, mode]'

In [22]:
data_file_list = ["pmnlstmn.txt", "pmn96cur.txt", "pmn9195.txt", "pmn8690.txt", "pmn8185.txt", "pmn7680.txt"]
data_list = []
for i in range(len(data_file_list)):
    path = os.path.join(abs_path, "FDA Registration Record", "510k", data_file_list[i])
    data_list.append(pd.read_csv(path, sep= "|", encoding="cp1252"))

DATA = pd.concat(data_list)
DATA = DATA.reset_index(drop=True)


In [23]:
FDA_INFO_510K = DATA[['KNUMBER', 'APPLICANT', "COUNTRY_CODE", "DATERECEIVED", 'DECISIONDATE', "DECISION", 'PRODUCTCODE', "STATEORSUMM", "THIRDPARTY", 'DEVICENAME']]
FDA_INFO_510K = FDA_INFO_510K.rename({'APPLICANT':'MANUFACTOR', "DEVICENAME":"MODELNAME"}, axis="columns")
FDA_INFO_510K = FDA_INFO_510K.replace(np.nan, "NA", regex=True)
FDA_INFO_510K['DATERECEIVED'] = pd.to_datetime(FDA_INFO_510K['DATERECEIVED'], errors = "coerce")
FDA_INFO_510K['DECISIONDATE'] = pd.to_datetime(FDA_INFO_510K['DECISIONDATE'], errors = "coerce")

In [24]:
import datetime as dt

In [25]:
# FDA_INFO_510K = FDA_INFO_510K.replace(np.nan, "NA", regex=True)
# FDA_INFO_510K["NAME"] = FDA_INFO_510K["MODELNAME"].str.cat(
#     FDA_INFO_510K["MANUFACTOR"].astype(str), sep="-")
# FDA_INFO_510K

In [26]:
import datetime as dt
filter_510k_kNumber = lambda df, knumber : df[df["KNUMBER"].str.lower() == knumber.lower()]
filter_510k_manufactor = lambda df, manufactor : df[df["MANUFACTOR"].str.lower().str.contains(manufactor.lower())]
filter_510k_countryCode = lambda df, countryCode : df[df["COUNTRY_CODE"].str.lower() == countryCode.lower()]
# mode = [before, in, after]
def filter_510k_dateReceived(df, year_mode):
    if year_mode[1] == 'before':
        result_df = df[df["DATERECEIVED"].dt.year < year_mode[0]]
    elif year_mode[1] == "in":
        result_df = df[df["DATERECEIVED"].dt.year == year_mode[0]]
    else:
        result_df = df[df["DATERECEIVED"].dt.year > year_mode[0]]
    return result_df
# mode = [before, in, after]
def filter_510k_decisionDate(df, year_mode):
    if year_mode[1] == 'before':
        result_df = df[df["DECISIONDATE"].dt.year < year_mode[0]]
    elif year_mode[1] == "in":
        result_df = df[df["DECISIONDATE"].dt.year == year_mode[0]]
    else:
        result_df = df[df["DECISIONDATE"].dt.year > year_mode[0]]
    return result_df
filter_510k_decision = lambda df, decision : df[df["DECISION"].str.lower() == decision.lower()]
filter_510k_productCode = lambda df, productCode : df[df["PRODUCTCODE"].str.lower() == productCode.lower()]
filter_510k_stateOrSum = lambda df, stateOrSum : df[df["STATEORSUMM"].str.lower() == stateOrSum.lower()]
filter_510k_thirdParty = lambda df, thirdParty : df[df["THIRDPARTY"].str.lower() == thirdParty.lower()]
filter_510k_modelName = lambda df, modelName: df[df["MODELNAME"].str.lower().str.contains(modelName.lower())]

func_list = [filter_510k_kNumber, filter_510k_manufactor, filter_510k_countryCode, filter_510k_dateReceived, filter_510k_decisionDate, filter_510k_decision, filter_510k_productCode, filter_510k_stateOrSum, filter_510k_thirdParty, filter_510k_modelName]

In [27]:
def filter_510k(df, knumber = None, manufactor = None, countryCode = None, dateReceived = None, decisionDate = None, decision = None, productCode = None, stateOrSum = None, thirdParty = None, ModelName = None):
    input_list = [knumber, manufactor, countryCode, dateReceived, decisionDate, decision, productCode, stateOrSum, thirdParty, ModelName]
    result_df = df
    for i in range(len(input_list)):
        if input_list[i] != None:
            result_df = func_list[i](result_df, input_list[i])
    return result_df

In [28]:
result = filter_510k(FDA_INFO_510K, thirdParty= "Y", )
result

,KNUMBER,MANUFACTOR,COUNTRY_CODE,DATERECEIVED,DECISIONDATE,DECISION,PRODUCTCODE,STATEORSUMM,THIRDPARTY,MODELNAME
242,K202456,Global Instrumentation LLC,US,2020-08-27,2020-12-29,SESE,DSH,Summary,Y,M5 Recorder
335,K203066,"Avanos Medical, Inc.",US,2020-10-09,2020-12-22,SESE,GXI,Summary,Y,COOLIEF Cooled Radiofrequency Kit Advanced
336,K203087,Philips Medical Systems DMC GmbH,DE,2020-10-13,2020-12-03,SESE,JAA,Summary,Y,CombiDiagnost R90
359,K203189,"Aerobiotix, Inc.",US,2020-10-27,2021-01-21,SESE,FRA,Summary,Y,AEROCURE-MD
363,K203209,AirPhysio Pty Ltd,AU,2020-10-30,2021-01-15,SESE,BWF,Summary,Y,AirPhysio Positive Expiratory (PEP) Device
...,...,...,...,...,...,...,...,...,...,...
82523,K993888,"A & D ENGINEERING, INC.",US,1999-11-16,1999-12-20,SESE,DXN,Statement,Y,A & D MEDICALLIFESOURCE US-779PC & UA-779T DIG...
82525,K993890,"NIHON SEIMITSU SOKKI CO., LTD.",US,1999-11-16,1999-12-01,SESE,DXN,Summary,Y,"DIGITAL BLOOD PRESSURE MONITOR, MODEL DS-181"
82566,K993944,RITA MEDICAL SYSTEMS,US,1999-11-19,1999-12-02,SESE,GEI,Summary,Y,"RITA MODEL 1500 ELECTROSURGICAL GENERATOR, POW..."
82567,K993945,"BIONIKE LABORATORIES, INC.",US,1999-11-19,1999-12-07,SESE,LCM,Summary,Y,BIONIKE AQ PHENCYCLIDINE (PCP) TEST


## analyzing functions:

In [29]:
# example
def analyze_num_of_df(df):
    return df.shape[0]

In [30]:

result_df = filter_510k(FDA_INFO_510K, thirdParty= "Y", )
result = analyze_num_of_df(result_df)
result

3126